In [134]:
import cv2
import dlib
import numpy as np
!pip uninstall opencv-python -y
# downgrade OpenCV a bit to use SIFT
!pip install opencv-contrib-python --force-reinstall
!pip install ffmpeg-python # for converting to video

import ffmpeg
import cv2
import numpy as np
import os
from numpy.linalg import svd, inv
import utils
%matplotlib inline
from matplotlib import pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached opencv_contrib_python-4.7.0.72-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (67.9 MB)
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.7.0.72
    Uninstalling opencv-contrib-python-4.7.0.72:
      Successfully uninstalled opencv-contrib-python-4.7.0.72
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imgaug 0.4.0 requires opencv-python, which is not installed.
dopamine-rl 4.0.6 requires opencv-python>=3.4.8.29, which is not installed.
tensorf

In [ ]:
import os
os.mkdir('video')

In [135]:
utils.video2imageFolder('Untitled.mov', 'video')

In [136]:
# read all the images
import os 
dir_frames = 'video'
filenames = []
filesinfo = os.scandir(dir_frames)

filenames = [f.path for f in filesinfo if f.name.endswith(".jpg")]
filenames.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

frameCount = len(filenames)
frameHeight, frameWidth, frameChannels = cv2.imread(filenames[0]).shape
frames = np.zeros((frameCount, frameHeight, frameWidth, frameChannels),dtype='uint8')

for idx, file_i in enumerate(filenames):
  frames[idx] = cv2.imread(file_i)

In [137]:
def warpImgs(face, body):
  #https://pysource.com/2019/05/28/face-swapping-explained-in-8-steps-opencv-with-python/
  face1 = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
  body1 = cv2.cvtColor(body, cv2.COLOR_BGR2GRAY)

  detector = dlib.get_frontal_face_detector()
  predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")

  landmarks = predictor(face1, detector(face1)[0])
  landmarks_points = [] 

  for n in range(68):
      x = landmarks.part(n).x
      y = landmarks.part(n).y
      landmarks_points.append((x, y))

  points = np.array(landmarks_points)
  subdiv = cv2.Subdiv2D(cv2.boundingRect(cv2.convexHull(points)))
  subdiv.insert(landmarks_points)
  triangles = subdiv.getTriangleList()
  triangles = np.array(triangles, dtype=np.int32)

  indexes_triangles = []

  def extract_index_nparray(arr):
      index = 0
      if arr[0]:
          index = arr[0][0]
      return index

  for triangle in triangles:
      pt1 = (triangle[0], triangle[1])
      pt2 = (triangle[2], triangle[3])
      pt3 = (triangle[4], triangle[5])

      index_pt1 = np.where((points == pt1).all(axis=1))
      index_pt1 = extract_index_nparray(index_pt1)
      index_pt2 = np.where((points == pt2).all(axis=1))
      index_pt2 = extract_index_nparray(index_pt2)
      index_pt3 = np.where((points == pt3).all(axis=1))
      index_pt3 = extract_index_nparray(index_pt3)

      if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
          vertices = [index_pt1, index_pt2, index_pt3]
          indexes_triangles.append(vertices)
 
  landmarks_2 = predictor(body1, detector(body1)[0])
  landmarks_points2 = []

  for n in range(68):
      x = landmarks_2.part(n).x
      y = landmarks_2.part(n).y
      landmarks_points2.append((x, y))

  convexhull2 = cv2.convexHull(np.array(landmarks_points2))

  warp_face = np.zeros((body.shape[0], body.shape[1], body.shape[2]), np.uint8)

  def get_landmark_one(triangle, landmark):
      pt1 = landmark[triangle[0]]
      pt2 = landmark[triangle[1]]
      pt3 = landmark[triangle[2]]

      (x, y, width, height) = cv2.boundingRect(np.array([pt1, pt2, pt3], np.int32))
      cropped_triangle = face[y: y+height, x: x+width]
      cropped_mask = np.zeros((height, width), np.uint8)

      points = np.array([[pt1[0]-x, pt1[1]-y], [pt2[0]-x, pt2[1]-y], [pt3[0]-x, pt3[1]-y]], np.int32)
      cv2.fillConvexPoly(cropped_mask, points, 255)

      return np.float32(points), cropped_mask, cropped_triangle

  for triangle in indexes_triangles:
      points, cropped_mask, cropped_triangle = get_landmark_one(triangle, landmarks_points)
      
      pt1 = landmarks_points2[triangle[0]]
      pt2 = landmarks_points2[triangle[1]]
      pt3 = landmarks_points2[triangle[2]]

      (x, y, width, height) = cv2.boundingRect(np.array([pt1, pt2, pt3], np.int32))
      cropped_mask2 = np.zeros((height,width), np.uint8)

      points2 = np.array([[pt1[0]-x, pt1[1]-y], [pt2[0]-x, pt2[1]-y], [pt3[0]-x, pt3[1]-y]], np.int32)
      cv2.fillConvexPoly(cropped_mask2, points2, 255)

      points2 = np.float32(points2)
      M = cv2.getAffineTransform(points, points2)
      warped_triangle = cv2.warpAffine(cropped_triangle, M, (width, height))
      warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=cropped_mask2)

      new_img = cv2.cvtColor(warp_face[y: y+height, x: x+width], cv2.COLOR_BGR2GRAY)

      masked_triangle = cv2.threshold(new_img, 1, 255, cv2.THRESH_BINARY_INV)
      warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=masked_triangle[1])

      warped_img = cv2.add(warp_face[y: y+height, x: x+width], warped_triangle)
      warp_face[y: y+height, x: x+width] = warped_img
    

  face_mask = np.zeros_like(body1)
  body_mask = cv2.fillConvexPoly(face_mask, convexhull2, 255)
  face_mask = cv2.bitwise_not(body_mask)

  maskless = cv2.bitwise_and(body, body, mask=face_mask)
  result = cv2.add(maskless, warp_face)

  (x, y, width, height) = cv2.boundingRect(convexhull2)

  seamlessclone = cv2.seamlessClone(result, body, body_mask, (int(((x*2)+width)/2), int(((y*2)+height)/2)), cv2.NORMAL_CLONE)
  return seamlessclone

In [139]:
# import shutil
# shutil.rmtree('warped')
os.mkdir('warped')

In [140]:
img1 = cv2.imread("emma.jpg")

# create your video (see tips)
for idx, file_i in enumerate(filenames):
    img = cv2.imread(file_i)
    img2 = warpImgs(img, img1)
            
    cv2.imwrite("warped/wp{:04d}.jpg".format(idx), img2)

In [142]:
utils.imageFolder2mpeg('warped', fps=30,  output_path='warpedVid.mpeg')